In [1]:
def normalize_name(s):
    s = s.replace('-',' ')
    s = s.replace('Stanislas','Stan').replace('Stan','Stanislas')
    s = s.replace('Alexandre','Alexander')
    s = s.replace('Federico Delbonis','Federico Del').replace('Federico Del','Federico Delbonis')
    s = s.replace('Mello','Melo')
    s = s.replace('Cedric','Cedrik')
    s = s.replace('Bernakis','Berankis')
    s = s.replace('Hansescu','Hanescu')
    s = s.replace('Teimuraz','Teymuraz')
    s = s.replace('Vikor','Viktor')
    s = s.rstrip()
    s = s.replace('Alex Jr.','Alex Bogomolov')
    s = s.title()
    sep = s.split(' ')
    return ' '.join(sep[:2]) if len(sep)>2 else s

In [2]:
# NEXT STEPS: assemble players win percentages from any set score (how well they do from one
# set up, one down, came back to split, gave up to split) throughout career; can represent 
# this as a beta distribution
# with prior as the average player of that current elo's probability with strength 5-10

# TO DO: assemble running tallies to track serve percentages over the past 52 weeks, specifically for years 2010-current
#        also, compare and contrast the "old" and "new" files


# 7/26 update: seem to have fixed some glitches in my previous adaptation of 538's elo method
# everything is very simple in elo_538.py; also now fixing the match order inside tournaments within atp_all_matches

# before: we improved from 9563 to 10000 matches with string manipulations

# my elo scores seem close to Jeff Sackman's when I set k=32 as constant; when using the 538 style of "cooling" k-factor,
# those elos are going to be inflated in comparison (but keep in mind, that's only different across methods)
AO = 'Australian Open'
USO = 'US Open'
COUNT = False

In [6]:
import pandas as pd
a = pd.read_csv("../tennis_data/atp_matches_2010.csv")
a.columns

Index([u'tourney_id', u'tourney_name', u'surface', u'draw_size',
       u'tourney_level', u'tourney_date', u'match_num', u'winner_id',
       u'winner_seed', u'winner_entry', u'winner_name', u'winner_hand',
       u'winner_ht', u'winner_ioc', u'winner_age', u'winner_rank',
       u'winner_rank_points', u'loser_id', u'loser_seed', u'loser_entry',
       u'loser_name', u'loser_hand', u'loser_ht', u'loser_ioc', u'loser_age',
       u'loser_rank', u'loser_rank_points', u'score', u'best_of', u'round',
       u'minutes', u'w_ace', u'w_df', u'w_svpt', u'w_1stIn', u'w_1stWon',
       u'w_2ndWon', u'w_SvGms', u'w_bpSaved', u'w_bpFaced', u'l_ace', u'l_df',
       u'l_svpt', u'l_1stIn', u'l_1stWon', u'l_2ndWon', u'l_SvGms',
       u'l_bpSaved', u'l_bpFaced'],
      dtype='object')

In [15]:
a.loc[0]

tourney_id                 2010-301
tourney_name               Auckland
surface                        Hard
draw_size                        28
tourney_level                     A
tourney_date               20100111
match_num                         1
winner_id                    103752
winner_seed                     NaN
winner_entry                    NaN
winner_name             Simon Greul
winner_hand                       R
winner_ht                       185
winner_ioc                      GER
winner_age                  28.7474
winner_rank                      60
winner_rank_points              737
loser_id                     103893
loser_seed                      NaN
loser_entry                       Q
loser_name            Paolo Lorenzi
loser_hand                        R
loser_ht                        183
loser_ioc                       ITA
loser_age                   28.0739
loser_rank                       86
loser_rank_points               577
score                       

In [3]:
import pandas as pd
import numpy as np
import datetime
import math
import elo_538 as elo
import re

atp_year_list = []
for i in xrange(1968,2018):
    atp_year_list.append(pd.read_csv("../tennis_data/atp_matches_{0}.csv".format(i)))
atp_all_matches = pd.concat(atp_year_list, ignore_index = True)

# Set up the pbp (point by point) column that will contain the pbp information for matches we have info on
atp_all_matches['pbp'] = [None]*len(atp_all_matches)
atp_all_matches = atp_all_matches.rename(columns={'winner_name':'w_name','loser_name':'l_name'})
atp_all_matches['w_name'] = [normalize_name(x) for x in atp_all_matches['w_name']]
atp_all_matches['l_name'] = [normalize_name(x) for x in atp_all_matches['l_name']]
atp_all_matches['tourney_name'] = ['Davis Cup' if 'Davis Cup' in s else s for s in atp_all_matches['tourney_name']]
atp_all_matches['tourney_name'] = [s.replace('Australian Chps.',AO).replace('Australian Open-2',AO).replace('U.S. National Chps.',USO) \
                                   for s in atp_all_matches['tourney_name']]
grand_slam_d = dict(zip(['Australian Open','Roland Garros','Wimbledon','US Open'],[1]*4))
atp_all_matches['is_gs'] = [name in grand_slam_d for name in atp_all_matches['tourney_name']]

# Get dates into the same format
atp_all_matches['tourney_date'] = [datetime.datetime.strptime(str(x), "%Y%m%d").date() for x in atp_all_matches['tourney_date']]
atp_all_matches['match_year'] = [x.year for x in atp_all_matches['tourney_date']]
atp_all_matches['match_month'] = [x.month for x in atp_all_matches['tourney_date']]
atp_all_matches['score'] = [re.sub(r"[\(\[].*?[\)\]]", "", str(s)) for s in atp_all_matches['score']] # str(s) fixes any nans
atp_all_matches['score'] = ['RET' if 'RET' in s else s for s in atp_all_matches['score']]
ret_d = dict(zip(['ABN','DEF','In Progress','RET','W/O',' RET',' W/O','nan'],[1]*8))
atp_all_matches = atp_all_matches.loc[[i for i in range(len(atp_all_matches)) if atp_all_matches['score'][i] not in ret_d]]
atp_all_matches = atp_all_matches.sort_values(by=['tourney_date','tourney_name','match_num'], ascending=True).reset_index()
del atp_all_matches['index']
atp_all_matches = atp_all_matches[['tourney_id','tourney_name','tourney_date','match_num','surface','is_gs','w_name','l_name',
                'winner_id','loser_id','best_of','match_year','match_month','score']]
print atp_all_matches.shape
atp_all_matches.head(2)

(159615, 14)


,tourney_id,tourney_name,tourney_date,match_num,surface,is_gs,w_name,l_name,winner_id,loser_id,best_of,match_year,match_month,score
0,1968-580,Australian Open,1968-01-19,1,Grass,True,Richard Coulthard,Max Senior,110023,107760,5,1968,1,12-10 7-5 4-6 7-5
1,1968-580,Australian Open,1968-01-19,2,Grass,True,John Brown,Ernie Mccabe,109803,106964,5,1968,1,6-3 6-2 6-4


In [26]:
# also can filter 'ABD', 'played and abandoned', 'DEF', etc
# for score in sorted(list(set(atp_all_matches['score']))):
#     print score

## Rating Systems

1. http://gobase.org/studying/articles/elo/
2. http://www.gamefaqs.com/boards/610657-dota-2/67994646
3. https://github.com/sublee/elo/blob/master/elo.py
4. http://stephenwan.net/thoughts/2012/10/02/elo-rating-system.html
5. https://deltadata.wordpress.com/2014/01/11/glicko-2-for-tennis-part-2-the-model/comment-page-1/#comment-192
6. https://github.com/sublee/glicko2/blob/master/glicko2.py
7. http://trueskill.org/
8. https://pypi.python.org/pypi/trueskill


In [27]:
### Make a dict of players with elo ratings ###
# now, adding three more surface-specific dictionaries for clay, grass, hard
players_list = np.union1d(atp_all_matches.w_name, atp_all_matches.l_name)
players_elo = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))]))
surface_elo = {}
for surface in ('Hard','Clay','Grass'):
    surface_elo[surface] = dict(zip(list(players_list), [elo.Rating() for __ in range(len(players_list))])) 

elo_1s, elo_2s = [],[]
surface_elo_1s, surface_elo_2s = [],[]
elo_obj = elo.Elo_Rater()

for i, row in atp_all_matches.iterrows():
    surface = row['surface']; is_gs = row['is_gs']
    # append elos, rate, update
    w_elo,l_elo = players_elo[row['w_name']],players_elo[row['l_name']]
    elo_1s.append(w_elo.value);elo_2s.append(l_elo.value)    
    elo_obj.rate_1vs1(w_elo,l_elo,is_gs,counts=COUNT)

    
    surface_elo_1s.append(surface_elo[surface][row['w_name']].value if surface in ('Hard','Clay','Grass') else w_elo.value)
    surface_elo_2s.append(surface_elo[surface][row['l_name']].value if surface in ('Hard','Clay','Grass') else l_elo.value)
    if surface in ('Hard','Clay','Grass'):
        new_elo1, new_elo2 = elo_obj.rate_1vs1(surface_elo[surface][row['w_name']],surface_elo[surface][row['l_name']],is_gs,counts=COUNT)

# add columns
atp_all_matches['w_elo'], atp_all_matches['l_elo'] = elo_1s, elo_2s
atp_all_matches['sw_elo'], atp_all_matches['sl_elo'] = surface_elo_1s, surface_elo_2s

In [30]:
rankings = sorted(zip(players_elo.keys(),players_elo.values()),key=lambda x: x[1].value,reverse=True)
#rankings

In [20]:
#None in set(atp_all_matches['tourney_id'])

In [31]:
# Combine all the matches that have pbp (point by point) information into one dataframe
# and clean up columns in preparation for mergin with all_atp_matches

pbp_matches_archive = pd.read_csv("../tennis_data/pbp_matches_atp_main_archive.csv")
pbp_matches_archive_old = pd.read_csv("../tennis_data/pbp_matches_atp_main_archive_old.csv")
pbp_matches_current = pd.read_csv("../tennis_data/pbp_matches_atp_main_current.csv")
pbp_matches = pd.concat([pbp_matches_archive_old.loc[:932],pbp_matches_archive,pbp_matches_current])
pbp_matches.winner = pbp_matches.winner - 1
pbp_matches = pbp_matches.reset_index(); del pbp_matches['index']
pbp_matches['w_name'] = np.where(pbp_matches['winner'] == 0, pbp_matches['server1'], pbp_matches['server2'])
pbp_matches['l_name'] = np.where(pbp_matches['winner'] == 0, pbp_matches['server2'], pbp_matches['server1'])
pbp_matches['w_name'] = [normalize_name(x) for x in pbp_matches['w_name']]
pbp_matches['l_name'] = [normalize_name(x) for x in pbp_matches['l_name']]
pbp_matches['date'] = pd.to_datetime(pbp_matches['date'])
pbp_matches['match_year'] = [x.year for x in pbp_matches['date']]
pbp_matches['match_month'] = [x.month for x in pbp_matches['date']]
pbp_matches['date'] = [x.date() for x in pbp_matches['date']]
pbp_matches['score'] = [re.sub(r"[\(\[].*?[\)\]]", "", s) for s in pbp_matches['score']]

print pbp_matches.shape
print len(set(pbp_matches['pbp'].values))

(12086, 16)
11877


In [14]:
# Now, just make a dictionary with each key as 'w_name'+'l_name'+'match_year'+'score' to connect pbp 
# strings to atp_all_matches
# replacing dashes with spaces accounted for over 1000 more pbp matches
# other discrepancies due to tiebreak scores, or matches in pbp being qualies, therefore
# not tour-level matches
# MUST REMOVE PARENTHESES TERMS FROM TIE-BREAKS IN SCORES

pbp_dict = {}; winner_dict = {}
for i in xrange(len(pbp_matches)):
    key = pbp_matches['w_name'][i] +' ' +  pbp_matches['l_name'][i] + ' ' \
        + str(pbp_matches['match_year'][i]) + ' ' + pbp_matches['score'][i]
    pbp_dict[key] = pbp_matches['pbp'][i]
    winner_dict[key] = pbp_matches['winner'][i]

In [15]:
# NOTE: TO FIX THIS, use the 'tourney_id' column (actually, can't...)
c = 0
pbps = []
winners = []
info = {}

for i in xrange(len(atp_all_matches)):
    key = atp_all_matches['w_name'][i] +' ' +  atp_all_matches['l_name'][i] + ' ' \
        +str(atp_all_matches['match_year'][i])+' '+atp_all_matches['score'][i]
    if key in pbp_dict:
        c += 1
        pbps.append(pbp_dict[key])
        winners.append(winner_dict[key])
        if key in info:
            print 'collision'
            print key
        info[key] = 1
    else:
        pbps.append('NA')
        winners.append('NA')
print c
atp_all_matches['pbp'] = pbps
atp_all_matches['winner'] = winners

collision
Janko Tipsarevic Kei Nishikori 2011 6-4 6-4
collision
Robin Soderling Michael Berrer 2011 6-3 7-6
collision
Juan Martin Kevin Anderson 2011 6-4 6-4
collision
Philipp Kohlschreiber Mikhail Youzhny 2011 6-4 6-2
collision
Philipp Kohlschreiber Olivier Rochus 2012 6-1 6-4
collision
Viktor Troicki Radek Stepanek 2012 2-6 6-4 6-3
collision
Gilles Simon Grigor Dimitrov 2012 6-3 6-3
collision
Alexandr Dolgopolov Gilles Simon 2012 6-3 6-4
collision
Fabio Fognini Tommy Haas 2013 6-2 6-4
collision
Richard Gasquet Florian Mayer 2013 6-3 7-6
collision
Novak Djokovic Rafael Nadal 2013 6-3 6-4
collision
Tomas Berdych Gael Monfils 2015 6-1 6-4
collision
Novak Djokovic Rafael Nadal 2015 6-3 6-3
10675


In [16]:
x = atp_all_matches
x = x[x['match_year']==2017]
#x[x['w_name']=='Diego Schwartzman']

In [17]:
i = 155926
key = atp_all_matches['w_name'][i] +' ' +  atp_all_matches['l_name'][i] + ' ' \
        +str(atp_all_matches['match_year'][i])+' '+atp_all_matches['score'][i]
key

'Leonardo Mayer Steve Darcis 2015 7-6 7-6 4-6 6-3'

In [29]:
# pbp_matches.loc[3007]

In [30]:
# #demo of match-keys I created 
# a lot of the inconsistencies are challenger-level events or even exhibition matches (see 2017)
# for i in xrange(len(pbp_matches)):
#     key = pbp_matches['w_name'][i] +' ' +  pbp_matches['l_name'][i] + ' ' \
#         + str(pbp_matches['match_year'][i])+ ' ' + pbp_matches['score'][i]
#     if key not in info:
#         print i,key

In [19]:
df = atp_all_matches[atp_all_matches['pbp']!='NA']
cols = df.columns.drop(['loser_id','winner_id'])
df = df[cols]
df = df.reset_index()
del df['index']
# switching perspective from w/l to p0/p1 for future predictions

In [20]:
df['p0_name'] = [df['l_name'][i] if df['winner'][i] else df['w_name'][i] for i in xrange(len(df))]
df['p1_name'] = [df['w_name'][i] if df['winner'][i] else df['l_name'][i] for i in xrange(len(df))]
df['p0_elo'] = [df['l_elo'][i] if df['winner'][i] else df['w_elo'][i] for i in xrange(len(df))]
df['p1_elo'] = [df['w_elo'][i] if df['winner'][i] else df['l_elo'][i] for i in xrange(len(df))]
df['p0_s_elo'] = [df['sl_elo'][i] if df['winner'][i] else df['sw_elo'][i] for i in xrange(len(df))]
df['p1_s_elo'] = [df['sw_elo'][i] if df['winner'][i] else df['sl_elo'][i] for i in xrange(len(df))]
df['elo_diff'] = [df['p0_elo'][i] - df['p1_elo'][i] for i in xrange(len(df))]
df['s_elo_diff'] = [df['p0_s_elo'][i] - df['p1_s_elo'][i] for i in xrange(len(df))]
df = df[['tourney_id','tourney_name','tourney_date','p0_name','p1_name','p0_elo','p1_elo','elo_diff','p0_s_elo',\
         'p1_s_elo','s_elo_diff','best_of','score','pbp','winner']]
df['tourney_name'] = [s if s==s else 'Davis Cup' for s in df['tourney_name']]

#df.to_csv('../my_data/elo_pbp_with_surface.csv')

In [33]:
# an example of a qualifying match in pbp_matches that will not be exported in elo_pbp.csv
z = pbp_matches[pbp_matches['server2']=='Jurgen Zopp']
z[z['server1']=='Jack Sock']

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,w_name,l_name,match_year,match_month
2492,3362769,2012-08-18,WinstonSalemOpen-ATPWinstonSalem,ATP,Main,Jack Sock,Jurgen Zopp,0,SRSSS;SSSS;RSSSS;SSSS;SRRSSS;SSSS;SSSRS;RSRSSR...,6-4 7-6,0,116,Jack Sock,Jurgen Zopp,2012,8


In [80]:
# the order of matches in this dataframe is incorrect
# furthermore, the elo seems to sometimes drop after winning...

# df = atp_all_matches
# sub = atp_all_matches[atp_all_matches['match_year']==2011]
# sub.loc[[i for i in sub.index if sub['w_name'][i]=='Robin Soderling' or sub['l_name'][i]=='Robin Soderling']]

# columns = ['tourney_id','tourney_name','tourney_date','match_num','surface','is_gs','w_name','l_name',
#                 'winner_id','loser_id','best_of','w_elo','l_elo','sw_elo','sl_elo','match_year','match_month','score']
# atp_all_matches = atp_all_matches[columns]
#atp_all_matches[atp_all_matches['w_name']=='Roger Federer']

In [21]:
df.shape

(10675, 15)

In [41]:
below = df[df['elo_diff']<0]
above = df[df['elo_diff']>=0]

print (len(below[below['winner']==1])+len(above[above['winner']==0]))/float(len(df))

below = df[df['s_elo_diff']<0]
above = df[df['s_elo_diff']>=0]

print (len(below[below['winner']==1])+len(above[above['winner']==0]))/float(len(df))

0.695374163253
0.690178839045
